In [17]:
!pip install yfinance pandas lxml matplotlib pandas_market_calendars --quiet

import pandas as pd
import yfinance as yf
import numpy as np
import os
import datetime as dt
from pandas.tseries.offsets import CustomBusinessDay
from pandas_market_calendars import get_calendar 

# Step 1: Get the SnP 500 Tickers from Wikipedia

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
constituents_table = payload[0]
df = constituents_table

# Some ticker symbols contain periods (e.g., BRK.B), which Yahoo Finance expects as a dash (BRK-B)
tickers = df['Symbol'].str.replace('.', '-', regex=False).tolist()

df.head(5)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [18]:
# Step 2: Define the date range for downloading data from yfinance
end_date = dt.datetime.now()
end_date = end_date - dt.timedelta(days=1)
start_date = end_date - dt.timedelta(days=30)  # Adjust as needed

# Step 3: Batch download Yahoo Finance data for all tickers at 1-hour intervals
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    interval='1h',
    group_by='ticker',
    progress=False
)

# Step 4: Convert the datetime index to Eastern Time
# If the index is naïve, we assume it is in UTC, then convert to America/New_York
if data.index.tz is None:
    data.index = data.index.tz_localize('UTC')
data.index = data.index.tz_convert('America/New_York')

# Example: Display the first few rows of AAPL
print(data['AAPL'].head(24))


Price                            Open        High         Low       Close  \
Datetime                                                                    
2025-02-06 09:30:00-05:00  231.339996  233.800003  230.779297  231.539993   
2025-02-06 10:30:00-05:00  231.585007  232.325607  230.425003  231.514999   
2025-02-06 11:30:00-05:00  231.509995  231.634995  230.899994  231.330002   
2025-02-06 12:30:00-05:00  231.345001  232.339996  231.029999  232.190002   
2025-02-06 13:30:00-05:00  232.190002  233.110001  232.080002  232.869995   
2025-02-06 14:30:00-05:00  232.889999  232.889999  231.895004  232.279999   
2025-02-06 15:30:00-05:00  232.289993  233.279999  231.929993  233.220001   
2025-02-07 09:30:00-05:00  232.514999  234.000000  229.880005  230.389999   
2025-02-07 10:30:00-05:00  230.339996  231.268005  228.899994  229.054993   
2025-02-07 11:30:00-05:00  229.085007  229.419998  228.195007  228.400604   
2025-02-07 12:30:00-05:00  228.389999  229.440002  228.363297  229.298904   

In [22]:

# Step 5: Add a new 'prev_close' column from previous business day and forward fill to next business day hourly
# intervals for all the tickers in the multi dimentional data frame

# 1. Define the NYSE trading calendar (to handle holidays)
nyse = get_calendar('NYSE')
trading_days = nyse.schedule(start_date=start_date, end_date=end_date)
# Convert to DatetimeIndex
trading_days = pd.DatetimeIndex(trading_days.index)
holidays = nyse.holidays().holidays  # List of NYSE holidays

# Custom business day offset
bday = CustomBusinessDay(holidays=holidays)

# 2. Adjusted function to calculate prev_close
def add_prev_close(df):
    # Get daily closing prices (aligned to trading days)
    daily_close = df['Close'].resample(bday).last()
    # Shift to previous trading day
    prev_day_close = daily_close.shift(1)
    # Forward-fill to hourly intervals
    df['Prev_Close'] = prev_day_close.reindex(df.index, method='ffill')
    return df

# 3. Process data (no grouping by day)
tickers = data.columns.get_level_values(0).unique()
processed_dfs = []

for ticker in tickers:
    df = data[ticker].copy()
    # Process entire history (not daily groups)
    df = add_prev_close(df)
    df.columns = pd.MultiIndex.from_product([[ticker], df.columns])
    processed_dfs.append(df)

data = pd.concat(processed_dfs, axis=1)

# Verify non-NaN results
print(data['INTC'][['Open','Close', 'Prev_Close']].head(50))


Price                           Open      Close  Prev_Close
Datetime                                                   
2025-02-06 09:30:00-05:00  19.530001  19.459999         NaN
2025-02-06 10:30:00-05:00  19.468399  19.366400         NaN
2025-02-06 11:30:00-05:00  19.365000  19.280001         NaN
2025-02-06 12:30:00-05:00  19.274401  19.370001         NaN
2025-02-06 13:30:00-05:00  19.360001  19.254999         NaN
2025-02-06 14:30:00-05:00  19.254999  19.285000         NaN
2025-02-06 15:30:00-05:00  19.290001  19.379999         NaN
2025-02-07 09:30:00-05:00  19.360001  19.145700   19.379999
2025-02-07 10:30:00-05:00  19.145000  19.072701   19.379999
2025-02-07 11:30:00-05:00  19.075001  19.129999   19.379999
2025-02-07 12:30:00-05:00  19.125000  19.125000   19.379999
2025-02-07 13:30:00-05:00  19.124701  19.075001   19.379999
2025-02-07 14:30:00-05:00  19.078899  19.075001   19.379999
2025-02-07 15:30:00-05:00  19.070000  19.105000   19.379999
2025-02-10 09:30:00-05:00  19.250000  19

In [25]:
# Step 6: Add a new 'Signal' column that will indicate 'Y' if the a ticker's first interval low is greater than x% 
# of the prev_close and the next 4hr interval lows are all greater than the first interval's low. Else it will be 'N'

THRESHOLD_PCT = 0.03  # 5%

# Get all tickers from the MultiIndex columns
tickers = data.columns.get_level_values(0).unique()

# Create a list to hold processed DataFrames
processed_dfs = []

for ticker in tickers:
    # Get the DataFrame for the current ticker
    df = data[ticker].copy()
    
    # Ensure the index is timezone-aware (NYSE time)
    df.index = df.index.tz_convert('America/New_York')
    
    # Initialize signal column with 'N'
    df['Signal'] = 'N'
    
    # Group by date (using NYSE trading days)
    df['date'] = df.index.date
    grouped = df.groupby('date')
    
    for date, group in grouped:
        # Skip days with <5 hours or missing prev_close
        if len(group) < 5 or group['Prev_Close'].isna().any():
            continue
            
        # Get first hour's low and prev_close
        first_low = group['Low'].iloc[0]
        prev_close = group['Prev_Close'].iloc[0]
        threshold = prev_close * (1 + THRESHOLD_PCT)
        
        # Condition 1: First low > 5% above prev_close
        if first_low > threshold:
            # Condition 2: Next 4 lows > first_low
            next_4_lows = group['Low'].iloc[1:5]
            if (next_4_lows > first_low).all():
                # Mark all rows in this day with 'Y'
                df.loc[group.index, 'Signal'] = 'Y'
    
    # Drop temporary 'date' column
    df = df.drop(columns=['date'])
    
    # Rebuild MultiIndex columns for the ticker
    df.columns = pd.MultiIndex.from_product([[ticker], df.columns])
    processed_dfs.append(df)

# Combine all tickers back into the original DataFrame
data = pd.concat(processed_dfs, axis=1)

# Verify the new column for INTC
print(data['INTC'][['Low', 'Prev_Close', 'Signal']].head(55))

Price                            Low  Prev_Close Signal
Datetime                                               
2025-02-06 09:30:00-05:00  19.379999         NaN      N
2025-02-06 10:30:00-05:00  19.320000         NaN      N
2025-02-06 11:30:00-05:00  19.250000         NaN      N
2025-02-06 12:30:00-05:00  19.260000         NaN      N
2025-02-06 13:30:00-05:00  19.250000         NaN      N
2025-02-06 14:30:00-05:00  19.219999         NaN      N
2025-02-06 15:30:00-05:00  19.270000         NaN      N
2025-02-07 09:30:00-05:00  19.030001   19.379999      N
2025-02-07 10:30:00-05:00  19.059999   19.379999      N
2025-02-07 11:30:00-05:00  19.049999   19.379999      N
2025-02-07 12:30:00-05:00  19.084801   19.379999      N
2025-02-07 13:30:00-05:00  19.059999   19.379999      N
2025-02-07 14:30:00-05:00  19.049999   19.379999      N
2025-02-07 15:30:00-05:00  19.059999   19.379999      N
2025-02-10 09:30:00-05:00  19.240000   19.105000      N
2025-02-10 10:30:00-05:00  19.680000   19.105000

In [26]:
# Step 7: Add a new 'Pct_Gain' column that will show the percentage gain calculated for all the tickers, for the days which the 
# Signal = 'Y' 

# Get all tickers from MultiIndex columns
tickers = data.columns.get_level_values(0).unique()

# Create a list to store processed DataFrames
processed_dfs = []

for ticker in tickers:
    # Get ticker data from MultiIndex
    df = data[ticker].copy()
    
    # Create date column (NYSE timezone-aware)
    df['date'] = df.index.tz_convert('America/New_York').date
    
    # Group by date to calculate daily metrics
    daily_metrics = df.groupby('date').agg({
        'Open': 'first',
        'Close': 'last',
        'Signal': lambda x: 'Y' if x.eq('Y').all() else 'N'
    })
    
    # Calculate percentage gain for 'Y' days
    daily_metrics['Pct_Gain'] = np.where(
        daily_metrics['Signal'] == 'Y',
        (daily_metrics['Close'] - daily_metrics['Open']) / daily_metrics['Open'] * 100,
        np.nan
    )
    
    # Merge back to hourly data
    df = df.merge(
        daily_metrics['Pct_Gain'],
        left_on='date',
        right_index=True,
        how='left'
    )
    
    # Cleanup and maintain MultiIndex
    df = df.drop(columns=['date'])
    df.columns = pd.MultiIndex.from_product([[ticker], df.columns])
    processed_dfs.append(df)

# Rebuild the entire DataFrame with new structure
data = pd.concat(processed_dfs, axis=1)


# Verify the new column for INTC
print(data['INTC'][['Signal', 'Pct_Gain']].head(20))

                          Signal  Pct_Gain
Datetime                                  
2025-02-06 09:30:00-05:00      N       NaN
2025-02-06 10:30:00-05:00      N       NaN
2025-02-06 11:30:00-05:00      N       NaN
2025-02-06 12:30:00-05:00      N       NaN
2025-02-06 13:30:00-05:00      N       NaN
2025-02-06 14:30:00-05:00      N       NaN
2025-02-06 15:30:00-05:00      N       NaN
2025-02-07 09:30:00-05:00      N       NaN
2025-02-07 10:30:00-05:00      N       NaN
2025-02-07 11:30:00-05:00      N       NaN
2025-02-07 12:30:00-05:00      N       NaN
2025-02-07 13:30:00-05:00      N       NaN
2025-02-07 14:30:00-05:00      N       NaN
2025-02-07 15:30:00-05:00      N       NaN
2025-02-10 09:30:00-05:00      N       NaN
2025-02-10 10:30:00-05:00      N       NaN
2025-02-10 11:30:00-05:00      N       NaN
2025-02-10 12:30:00-05:00      N       NaN
2025-02-10 13:30:00-05:00      N       NaN
2025-02-10 14:30:00-05:00      N       NaN


In [27]:
# Step 8: Display all the tickers with days having Signal = 'Y'

# Get all tickers with 'signal' column
tickers = data.columns.get_level_values(0).unique()

# Filter rows where signal is 'Y' for any ticker
filtered_rows = []

for ticker in tickers:
    # Get rows where current ticker's signal is 'Y'
    mask = data[(ticker, 'Signal')] == 'Y'
    ticker_y = data[ticker][mask]
    
    if not ticker_y.empty:
        # Add ticker name as column
        ticker_y = ticker_y.copy()
        ticker_y['Ticker'] = ticker
        filtered_rows.append(ticker_y)

# Combine all results
if filtered_rows:
    result_df = pd.concat(filtered_rows).reset_index()
    # Reorder columns
    cols = ['Datetime', 'Ticker'] + [c for c in result_df.columns if c not in ['Datetime', 'Ticker']]
    result_df = result_df[cols]
    print(f"Found {len(result_df)} matching rows:")
    display(result_df)
else:
    print("No rows with signal = 'Y' found")

Found 259 matching rows:


,Datetime,Ticker,Open,High,Low,Close,Volume,Prev_Close,Signal,Pct_Gain
0,2025-02-18 09:30:00-05:00,INTC,24.605000,26.240000,24.490999,26.089899,94841484,23.610001,Y,11.48141
1,2025-02-18 10:30:00-05:00,INTC,26.080000,26.209999,25.695000,25.915001,26492202,23.610001,Y,11.48141
2,2025-02-18 11:30:00-05:00,INTC,25.910000,26.250000,25.639999,25.714899,22623144,23.610001,Y,11.48141
3,2025-02-18 12:30:00-05:00,INTC,25.715000,26.209999,25.715000,26.150000,15898605,23.610001,Y,11.48141
4,2025-02-18 13:30:00-05:00,INTC,26.150000,26.559700,26.014999,26.555000,26299210,23.610001,Y,11.48141
...,...,...,...,...,...,...,...,...,...,...
254,2025-02-10 11:30:00-05:00,NUE,137.544998,138.570007,137.059998,138.462006,378574,130.220001,Y,0.57748
255,2025-02-10 12:30:00-05:00,NUE,138.549896,139.800003,138.369995,138.520004,655846,130.220001,Y,0.57748
256,2025-02-10 13:30:00-05:00,NUE,138.520004,139.029999,137.419998,137.699997,412877,130.220001,Y,0.57748
257,2025-02-10 14:30:00-05:00,NUE,137.710007,138.500000,137.520004,137.529999,382024,130.220001,Y,0.57748
